# Hike Planning


In [ ]:
import numpy as np
import copy

# Problem modelling imports
from docplex.mp.model import Model

# Qiskit imports
from qiskit import BasicAer
# from qiskit.utils.algorithm_globals import algorithm_globals
from qiskit_optimization.problems.variable import VarType
from qiskit_optimization.converters.quadratic_program_to_qubo import QuadraticProgramToQubo
from qiskit_optimization.translators import from_docplex_mp

from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_optimization.algorithms import (
    CplexOptimizer,
    MinimumEigenOptimizer,
    RecursiveMinimumEigenOptimizer,
    SolutionSample,
    OptimizationResultStatus,
)
from qiskit_optimization import QuadraticProgram
from qiskit.visualization import plot_histogram
from typing import List, Tuple
import numpy as np
from qiskit.quantum_info import Statevector
from qiskit.opflow import PauliExpectation, StateFn, CircuitSampler

In [ ]:
def create_problem(mu: np.array, sigma: np.array, total: int = 3) -> QuadraticProgram:
    """Solve the quadratic program using docplex."""

    mdl = Model()
    x = [mdl.binary_var("x%s" % i) for i in range(len(sigma))]

    objective = mdl.sum([mu[i] * x[i] for i in range(len(mu))])
    objective += mdl.sum(
        [sigma[i, j] * x[i] * x[j] for i in range(len(mu)) for j in range(len(mu))]
    )
    mdl.minimize(objective)
#     cost = mdl.sum(x)
#     mdl.add_constraint(cost == total)

    qp = from_docplex_mp(mdl)
    return qp


def relax_problem(problem) -> QuadraticProgram:
    """Change all variables to continuous."""
    relaxed_problem = copy.deepcopy(problem)
    for variable in relaxed_problem.variables:
        variable.vartype = VarType.CONTINUOUS

    return relaxed_problem

In [ ]:
# h = np.array([396, 752, 1204, 1177])
# d = np.array([1040, 2480, 7960, 3230])
# T = 5
# M = 1000.0

h = np.array([870, 2501, 4478, 1777, 2970])
d = np.array([2700, 6500, 15000, 100, 10000])
T = 13.5
M = 1000

t = h/(M/2.0) + d/(5.0*M)
print(t)
Q = np.outer(t, t)
print(Q)
b = -2.0 * T * t
# print(b)
# Q = Q + np.diag(b)
# print(Q)

In [ ]:
import itertools

values = []
print("x, cost")
for x in itertools.product([0, 1], repeat=t.size):
    values.append((x, (T-np.asarray(x).dot(t))**2))
    print(*values[-1])

optimal = min(values, key=lambda value: value[1])
print("Optimal solution:", *optimal)

In [ ]:
qubo = create_problem(b, Q)
qubo

## A. Constructing the Ising Hamiltonian

Math! No tips here.

In [ ]:
H, offset = qubo.to_ising()
print("offset: {}".format(offset))
print("operator:")
print(H)

In [ ]:
H.to_matrix()

In [ ]:
exact = NumPyMinimumEigensolver().compute_minimum_eigenvalue(H)
exact.eigenstate

In [ ]:
statevector = exact.eigenstate.primitive
print("Optimal state from Hamiltonian:")
print(statevector.probabilities_dict())
print("Value:")
print(exact.eigenvalue)

print("\nCompare to orignal solution:")
print(optimal)

## B. Finding the minimum

### Evaluating the cost function

First, construct the QAOA ansatz and then write a function to evaluate the energy. Given a quantum circuit, you can evaluate the expectation value as

In [ ]:
from qiskit.circuit.library import QAOAAnsatz
from qiskit.circuit.library import QAOAAnsatz, GroverOperator, RealAmplitudes

p = 3
#qaoa = QAOAAnsatz(H, reps=p)
qaoa = RealAmplitudes(5, reps=p, entanglement='linear')

#qaoa.draw("mpl", style="iqx")
qaoa.decompose() #.draw("mpl", style="iqx")

from qiskit import transpile

#transpile(qaoa, basis_gates=["h", "rx", "rz", "rzz"]) #.draw("mpl", style="iqx")
transpiled = qaoa.decompose().decompose().decompose()
#transpile(qaoa, basis_gates=["h", "rx", "rz", "rzz"]).draw("mpl", style="iqx")
transpile(transpiled, backend, optimization_level=3).draw("mpl", style="iqx")

In [ ]:
def exact_energy(theta):
    bound = qaoa.bind_parameters(theta)
    return np.real(Statevector(bound).expectation_value(H.primitive))

In [ ]:
delta_t = 0.75
p_ = 10

annealing_init = np.zeros(2*p_)

for i in range(p_):
    annealing_init[i] = (1-(i+1)/p_) * delta_t
    annealing_init[i+p_] = (i+1)/p_ * delta_t
annealing_init

In [ ]:
exact_energy(annealing_init)

### Optimize the energy

Now optimize the energy. You could for instance use one of Qiskit's built-in optimizers as

In [ ]:
final_state = Statevector(qaoa.bind_parameters(result.x))
probabilities = final_state.probabilities_dict()

plot_histogram(probabilities)

# The rest is not yet runnable

In [ ]:
from qiskit.algorithms.optimizers import L_BFGS_B, SPSA, COBYLA, CG, SLSQP

optimizer = SLSQ()

#initial_parameters = np.arange(qaoa.num_parameters)
initial_parameters = annealing_init
print(qaoa.num_parameters)
result = optimizer.minimize(exact_energy, initial_parameters)

print(result)

In [ ]:
# X, Y = np.meshgrid(betas, gammas)
# plt.figure(figsize=(12, 6))
# plt.contourf(X, Y, landscape)
# plt.xlabel(r"$\beta$")
# plt.ylabel(r"$\gamma$")
# plt.scatter(result.x[0], result.x[1], s=100, color="crimson")
# plt.colorbar()

In [ ]:
print("Number of circuit executions:", result.nfev)

In [ ]:
from qiskit.visualization import plot_histogram

final_state = Statevector(qaoa.bind_parameters(result.x))
probabilities = final_state.probabilities_dict()

plot_histogram(probabilities)

**Expected result** The optimal solutions should be at least among the 5 most likely states and have a probability distinctively larger than $1 / 2^\text{n}$, where $n$ is the number of qubits.

### Warmstarting

In [ ]:
algorithm_globals.random_seed = 12345

quantum_instance = QuantumInstance(
    BasicAer.get_backend("statevector_simulator"),
    seed_simulator=algorithm_globals.random_seed,
    seed_transpiler=algorithm_globals.random_seed,
)


from qiskit_optimization.algorithms import WarmStartQAOAOptimizer, CplexOptimizer
qaoa_mes = QAOA(quantum_instance=quantum_instance,
                optimizer=SPSA(maxiter=3),
                reps=3,
               )
ws_qaoa = WarmStartQAOAOptimizer(
    pre_solver=CplexOptimizer(), relax_for_pre_solver=True, qaoa=qaoa_mes, epsilon=0.1
)
ws_result = ws_qaoa.solve(qubo)
ws_result

In [ ]:
sol = CplexOptimizer().solve(qubo)
c_stars = sol.samples[0].x
c_stars

from qiskit import QuantumCircuit

thetas = [2 * np.arcsin(np.sqrt(c_star)) for c_star in c_stars]

init_qc = QuantumCircuit(len(Q))
for idx, theta in enumerate(thetas):
    init_qc.ry(theta, idx)

init_qc.draw(output="mpl")

from qiskit.circuit import Parameter

beta = Parameter("β")

ws_mixer = QuantumCircuit(len(Q))
for idx, theta in enumerate(thetas):
    ws_mixer.ry(-theta, idx)
    ws_mixer.rz(-2 * beta, idx)
    ws_mixer.ry(theta, idx)

ws_mixer.draw(output="mpl")

ws_qaoa_mes = QAOA(
    quantum_instance=quantum_instance,
    initial_state=init_qc,
    mixer=ws_mixer,
    initial_point=[0.0, 1.0],
)
ws_qaoa = MinimumEigenOptimizer(ws_qaoa_mes)
ws_qaoa_result = ws_qaoa.solve(qubo)
print(ws_qaoa_result)


### Recursive QAOA

In [ ]:
from qiskit_optimization.algorithms import RecursiveMinimumEigenOptimizer
from qiskit.algorithms.optimizers import L_BFGS_B, SPSA, COBYLA, CG, SLSQP

#qaoa = QAOAAnsatz(H, reps=p)
qaoa = RealAmplitudes(5, reps=5, entanglement='linear')
qaoa.decompose()
transpiled = qaoa.decompose().decompose().decompose()
transpile(transpiled, backend, optimization_level=3).draw("mpl", style="iqx")

# TODO: maybe this is set wrong
p = qaoa.num_parameters

initial_parameters = np.arange(2*p)
#initial_parameters = annealing_init

qaoa_mes = QAOA(quantum_instance=quantum_instance,
                reps=p,
                optimizer=SLSQP(),
                initial_point=initial_parameters)
qaoa_opt = MinimumEigenOptimizer(qaoa_mes)   # using QAOA

rqaoa = RecursiveMinimumEigenOptimizer(optimizer=qaoa_opt, min_num_vars=1)
rqaoa_result = rqaoa.solve(qubo)

In [ ]:
print(rqaoa_result)

final_state = Statevector(qaoa.bind_parameters(qaoa_result.x))
probabilities = final_state.probabilities_dict()
plot_histogram(probabilities)

### CVaR expectation

Update your energy evaluation such that only the best $\alpha \in (0, 1]$ fraction of the shots contribute to the expectation value.

In [ ]:
from qiskit.circuit.library import RealAmplitudes
from qiskit.algorithms.optimizers import COBYLA, GradientDescent, CG, QNSPSA
from qiskit.algorithms import NumPyMinimumEigensolver, VQE
from qiskit.opflow import PauliExpectation, CVaRExpectation
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import LinearEqualityToPenalty
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.translators import from_docplex_mp
from qiskit import execute, Aer
from qiskit.utils import algorithm_globals

import numpy as np
import matplotlib.pyplot as plt
from docplex.mp.model import Model

from qiskit.opflow.gradients import Gradient, NaturalGradient, QFI, Hessian

In [ ]:
# set classical optimizer
maxiter = 600
#optimizer = CG()
optimizer = QNSPSA(QNSPSA.get_fidelity(qaoa), maxiter=maxiter)

# set variational ansatz
# ansatz = RealAmplitudes(n, reps=1)
ansatz = qaoa
m = ansatz.num_parameters

# set backend
backend_name = "qasm_simulator"  # use this for QASM simulator
# backend_name = 'aer_simulator_statevector'  # use this for statevector simlator
backend = Aer.get_backend(backend_name)

# run variational optimization for different values of alpha
alphas = [1.0, 0.50, 0.25]  # confidence levels to be evaluated

# solve classically as reference
opt_result = MinimumEigenOptimizer(NumPyMinimumEigensolver()).solve(qubo)
opt_result

In [ ]:
print(offset)
print(T**2)

In [ ]:
# dictionaries to store optimization progress and results
objectives = {alpha: [] for alpha in alphas}  # set of tested objective functions w.r.t. alpha
results = {}  # results of minimum eigensolver w.r.t alpha

# callback to store intermediate results
def callback(i, params, obj, stddev, alpha):
    # we translate the objective from the internal Ising representation
    # to the original optimization problem
    objectives[alpha] += [obj + offset]


# loop over all given alpha values
for alpha in alphas:

    # initialize CVaR_alpha objective
    cvar_exp = CVaRExpectation(alpha, PauliExpectation())
    cvar_exp.compute_variance = lambda x: [0]  # to be fixed in PR #1373

    # initialize VQE using CVaR
    vqe = VQE(
        expectation=None,
        optimizer=optimizer,
        ansatz=ansatz,
        quantum_instance=backend,
        callback=lambda i, params, obj, stddev: callback(i, params, obj, stddev, alpha),
    )

    # initialize optimization algorithm based on CVaR-VQE
    opt_alg = MinimumEigenOptimizer(vqe)

    # solve problem
    results[alpha] = opt_alg.solve(qubo)

    # print results
    print("alpha = {}:".format(alpha))
    print(results[alpha])
    print()



In [ ]:
from scipy.ndimage.filters import gaussian_filter1d

# plot resulting history of objective values
plt.figure(figsize=(10, 5))
plt.plot([0, maxiter], [opt_result.fval, opt_result.fval], "r--", linewidth=2, label="optimum")
for alpha in alphas:
    #plt.plot(objectives[alpha], label="alpha = %.2f" % alpha, linewidth=2)
    plt.plot(gaussian_filter1d(objectives[alpha], sigma=2), label="alpha = %.2f" % alpha, linewidth=2)

plt.legend(loc="lower right", fontsize=14)
plt.xlim(0, maxiter)
plt.xticks(fontsize=14)
plt.xlabel("iterations", fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel("objective value", fontsize=14)
plt.show()

In [ ]:
# evaluate final optimal probability for each alpha
probs = {}
for alpha in alphas:
    s = 0
    if backend_name == "qasm_simulator":
        counts = results[alpha].min_eigen_solver_result.eigenstate
        shots = sum(counts.values())
        for key, val in counts.items():
            probs[key] = val / shots
            s += probs[key]
    else:
        probabilities = np.abs(results[alpha].min_eigen_solver_result.eigenstate)**2
    print(s)

In [ ]:
plot_histogram(probs)

## C. Circuit optimization

Now we'll transpile the circuit for the ``ibmq_quito`` device.

In [ ]:
from qiskit.test.mock import FakeQuito
backend = FakeQuito()

# or for the actual device, if you have an IBM Quantum Experience account
# from qiskit import IBMQ
# IBMQ.load_account()
# provider = IBMQ.get_provider(group="open")
# backend = provider.get_backend("ibmq_quito")

### Pulse efficient decomposition

*Note: This works correctly only for a circuit with bound parameters.*

You can draw a pulse schedule as follows.

In [ ]:
from qiskit import transpile, schedule, QuantumCircuit

circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)

transpiled = transpile(circuit, backend)

schedule(transpiled, backend).draw()